In [1]:
!pip install ctgan
!pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstallin

In [2]:
from ctgan import CTGAN 

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/club-churn.csv')
#filling missing value with mean
dataset['points_in_wallet'].fillna(value=dataset['points_in_wallet'].mean(), inplace=True)
dataset['avg_frequency_login_days'] = pd.to_numeric(dataset['avg_frequency_login_days'], errors='coerce').fillna(0)
#transform data into numeric value
dataset['gender']=le.fit_transform(dataset['gender'])
dataset['region_category']=le.fit_transform(dataset['region_category'])
dataset['membership_category']=le.fit_transform(dataset['membership_category'])
dataset['joined_through_referral']=le.fit_transform(dataset['joined_through_referral'])
dataset['preferred_offer_types']=le.fit_transform(dataset['preferred_offer_types'])
dataset['medium_of_operation']=le.fit_transform(dataset['medium_of_operation'])
dataset['internet_option']=le.fit_transform(dataset['internet_option'])
dataset['used_special_discount']=le.fit_transform(dataset['used_special_discount'])
dataset['offer_application_preference']=le.fit_transform(dataset['offer_application_preference'])
dataset['past_complaint']=le.fit_transform(dataset['past_complaint'])
dataset['complaint_status']=le.fit_transform(dataset['complaint_status'])
dataset['feedback']=le.fit_transform(dataset['feedback'])


dataset=dataset.dropna()
dataset=dataset.drop(['security_no','joining_date','referral_id','last_visit_time'],axis=1)

df1=dataset[dataset['churn_risk_score']==1]
df=dataset[dataset['churn_risk_score']==0]


batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)



Epoch 1, Loss G:  0.0510,Loss D: -0.0293
Epoch 2, Loss G:  0.0121,Loss D: -0.0685
Epoch 3, Loss G: -0.0684,Loss D: -0.0897
Epoch 4, Loss G: -0.2031,Loss D: -0.0907
Epoch 5, Loss G: -0.3524,Loss D: -0.1290
Epoch 6, Loss G: -0.4722,Loss D: -0.1726
Epoch 7, Loss G: -0.5987,Loss D: -0.1668
Epoch 8, Loss G: -0.6480,Loss D: -0.1410
Epoch 9, Loss G: -0.6705,Loss D: -0.0651
Epoch 10, Loss G: -0.6879,Loss D:  0.0019
Epoch 11, Loss G: -0.6847,Loss D: -0.0366
Epoch 12, Loss G: -0.6445,Loss D: -0.0644
Epoch 13, Loss G: -0.5882,Loss D: -0.0501
Epoch 14, Loss G: -0.5581,Loss D: -0.0201
Epoch 15, Loss G: -0.5222,Loss D: -0.0415
Epoch 16, Loss G: -0.4615,Loss D: -0.0625
Epoch 17, Loss G: -0.4683,Loss D: -0.0008
Epoch 18, Loss G: -0.4599,Loss D: -0.0359
Epoch 19, Loss G: -0.5326,Loss D:  0.0349
Epoch 20, Loss G: -0.6549,Loss D:  0.0831
Epoch 21, Loss G: -0.7784,Loss D:  0.1992
Epoch 22, Loss G: -0.9263,Loss D:  0.2552
Epoch 23, Loss G: -1.0131,Loss D:  0.2369
Epoch 24, Loss G: -1.0093,Loss D:  0.2290
E

In [3]:
n_generated_data = 3032
generated_df = model.sample(n_generated_data)

In [4]:
#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])
data

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
2,44,0,1,2,2,1,2,2,14,516.160000,21027.000000,22.000000,500.690000,0,1,1,3,3,1
3,37,1,0,2,2,1,2,1,11,53.270000,25239.560000,6.000000,567.660000,0,1,1,4,3,1
4,31,0,0,2,1,0,3,1,20,113.130000,24483.660000,16.000000,663.060000,0,1,1,2,3,1
7,42,1,3,2,0,0,1,0,19,429.110000,44554.820000,24.000000,568.080000,0,1,1,4,2,1
9,45,0,1,2,1,1,0,2,10,97.310000,19244.160000,28.000000,706.230000,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,58,1,0,3,1,0,3,2,-3,7.677190,1476.126189,-0.190711,955.014534,1,1,1,0,1,0
3028,57,0,1,5,2,2,2,1,4,88.355517,12368.469244,19.048517,830.473639,1,1,0,3,3,0
3029,12,1,1,4,1,2,2,2,7,61.888965,14611.857620,19.820630,851.755446,0,1,1,1,8,0
3030,16,0,0,3,2,2,2,2,20,36.404156,18656.804366,16.608175,747.686750,1,0,1,3,2,0


In [8]:
# Do Classification Proccess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import TomekLinks

data=data.dropna()
data=data.fillna(0)

X=data.drop(['churn_risk_score'],axis=1)
y=data["churn_risk_score"]

#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
X, y = ncr.fit_resample(X, y)

#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores




# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))

[0.9813014251429306, 0.9809079149070552]
F1 score: 0.944
STD F1 Score: 0.004
Recall: 0.952
Specitifity: 0.946
AUC ROC: 0.982
G-Mean: 0.949
